### Week 10 - text style transfer

Hello, sitzen class A.412C!

Based on your browser search history, we conclude that you have an above average skill in natural language processing. In our benevolence, we give you a chance to contribute your skills to upholding the happiest society in the universe. Are you up to the task?

As you know, our most recent breakthrough was replacing 97% restaurant workers with BFGHQBERT+++ autonomous food dispensers.

Yet a some radical elements failed to recognize the greater good that we brought them. They mistakenly voice their ignorant opinions about our new INGSOC-approved restaurants, brining dangerous doubt to the minds of our loyal citzens.

Surely you cannot tolerate such infidelity! Our loyal citzens demand that you rectify their mistake. _You must build a model that will automatically improve their ignorant thoughts and replace them with the thoughts they should actually have._

Attached below are the INGSOC-approved datasets for ignorant and correct thoughts. The scientific terminology is for wrong opinions and correct opinions is "negative" and "positive", respectively.

Respond within 7 days or you will lose 3.7629 citzenship points.

![img](https://ih1.redbubble.net/image.1254830934.9884/poster,504x498,f8f8f8-pad,400x240,f8f8f8.jpg)

In [ ]:
!pip install -q transformers
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.train.0 -O train_negative
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.train.1 -O train_positive
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.dev.0 -O dev_negative
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.dev.1 -O dev_positive

In [ ]:
!head -n 5 ./dev_positive
!echo
!head -n 5 ./dev_negative

staff behind the deli counter were super nice and efficient !
love this place !
the staff are always very nice and helpful .
the new yorker was amazing .
very ny style italian deli .

ok never going back to this place again .
easter day nothing open , heard about this place figured it would ok .
the host that walked us to the table and left without a word .
it just gets worse .
the food tasted awful .


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cpu':
    print("Fine-tuning BERT without an accelerator is not party-approved.")

### Part 1: Masked language model

Attached below you can find the INGSOC-compliant training code that fine-tunes a BERT model for Masked Language Modeling.

You shall use this model to generate positive replacements for negative tokens.

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_mlm_positive = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True).to(device).train(True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

print("Preparing the training data...")
dataset = LineByLineTextDataset(
    file_path="./train_positive", tokenizer=tokenizer, block_size=128)

print("Dataset ready!")

trainer = Trainer(
    model=bert_mlm_positive, train_dataset=dataset, 
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15),
    args=TrainingArguments(
        output_dir="./bert_mlm_positive", overwrite_output_dir=True,
        num_train_epochs=1, per_device_train_batch_size=32,
        save_steps=1_000, save_total_limit=2),
)

trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


Preparing the training data...
Dataset ready!


Step,Training Loss
500,1.446300
1000,1.166000
1500,1.130200
2000,1.072800
2500,1.077600
3000,1.048100
3500,1.056300
4000,1.043300
4500,1.007800
5000,1.005000


TrainOutput(global_step=8354, training_loss=1.0444445090628367, metrics={'train_runtime': 1469.2923, 'train_samples_per_second': 5.686, 'total_flos': 3284256698954208, 'epoch': 1.0})

In [ ]:
# <Build and train a MLM for incorrect opinions>

bert_mlm_negative = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True).to(device).train(True)
print("Preparing the training data...")
dataset = LineByLineTextDataset(
    file_path="./train_negative", tokenizer=tokenizer, block_size=128)

print("Dataset ready!")

trainer = Trainer(
    model=bert_mlm_negative, train_dataset=dataset, 
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15),
    args=TrainingArguments(
        output_dir="./bert_mlm_negative", overwrite_output_dir=True,
        num_train_epochs=1, per_device_train_batch_size=32,
        save_steps=1_000, save_total_limit=2),
)

trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm

Preparing the training data...
Dataset ready!


Step,Training Loss
500,1.499500
1000,1.244700
1500,1.185100
2000,1.152500
2500,1.133400
3000,1.109500
3500,1.080500
4000,1.096800
4500,1.082700
5000,1.057200


TrainOutput(global_step=5525, training_loss=1.1531272045843202, metrics={'train_runtime': 1032.1273, 'train_samples_per_second': 5.353, 'total_flos': 2399611589197200, 'epoch': 1.0})

### Part 2: Replace tokens

You can now use the two masked language models to align user opinions. You can do so with the following steps:

1. Find tokens where the ratio $(P_{positive}(x) + \epsilon) / (P_{negative}(x) + \epsilon)$ is the smallest
2. Replace those tokens with one of $k$ most likely tokens according to $P_{positive}(x)$.
3. Rinse, repeat

You can find the full procedure at https://arxiv.org/abs/2010.01054

In [ ]:
best_positive_checkpoint = './bert_mlm_positive/checkpoint-8000'
best_negative_checkpoint = './bert_mlm_negative/checkpoint-5000'
bert_mlm_positive = BertForMaskedLM.from_pretrained(best_positive_checkpoint, return_dict=True).to(device)
bert_mlm_negative = BertForMaskedLM.from_pretrained(best_negative_checkpoint, return_dict=True).to(device)

In [ ]:
bert_mlm_positive.train(False)
bert_mlm_negative.train(False);

In [ ]:
sentence = "great wings and decent drinks but the wait staff is horrible !"

In [ ]:
batch = tokenizer([f"great wings and decent drinks but the wait staff is horrible !"],
                  padding=True, truncation=True, return_tensors="pt")

In [ ]:
batch.to(device)

{'input_ids': tensor([[  101,  2307,  4777,  1998, 11519,  8974,  2021,  1996,  3524,  3095,
          2003,  9202,   999,   102]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
tokenizer.decode(batch['input_ids'][0])

'[CLS] great wings and decent drinks but the wait staff is horrible! [SEP]'

In [ ]:
bert_mlm_positive(**batch)['logits'].shape

torch.Size([1, 14, 30522])

In [ ]:
mask_position = batch['input_ids'][0].tolist().index(103)

In [ ]:
mask_position

11

In [ ]:
logits = bert_mlm_positive(**batch)['logits'][0, 11].detach().cpu().numpy()

In [ ]:
np.argsort(-logits)[:5]

array([ 2307, 12476,  6429, 10392,  5151])

In [ ]:
id_to_vocab = {i: token for token, i in tokenizer.vocab.items()}

In [ ]:
for index in np.argsort(-logits)[:5]:
  print(id_to_vocab[index])

great
awesome
amazing
fantastic
outstanding


In [ ]:
lsm = nn.Softmax(dim=2)

In [ ]:
def get_replacements(sentence: str, num_tokens, k_best, epsilon=1e-3):
  """
  - split the sentence into tokens using the INGSOC-approved BERT tokenizer
  - find :num_tokens: tokens with the highest ratio (see above)
  - replace them with :k_best: words according to bert_mlm_positive
  :return: a list of all possible strings (up to k_best * num_tokens)
  """
  batch = tokenizer([sentence], padding=True, truncation=True, return_tensors="pt").to(device)
  logits_p = bert_mlm_positive(**batch)['logits'].detach().cpu()
  probs_p = torch.max(lsm(logits_p), dim=2).values.squeeze(0).numpy()
  logits_n = bert_mlm_negative(**batch)['logits'].detach().cpu()
  probs_n = torch.max(lsm(logits_n), dim=2).values.squeeze(0).numpy()
  ratio = (probs_p + epsilon) / (probs_n + epsilon)
  ratio = ratio[1:-1]
  n = np.argsort(ratio)
  result = []
  for i in n[:num_tokens]:
    logits = logits_p[0, i + 1]
    for index in np.argsort(-logits)[:k_best].numpy():
      ids = batch['input_ids'][0].detach().cpu().numpy()
      ids = ids[1:-1]
      ids[i] = index
      s = tokenizer.decode(ids)
      result.append(s)
  return result

In [ ]:
get_replacements("great wings and decent drinks but the wait staff is horrible !",
                 num_tokens=1, k_best=3)

['great wings and decent drinks but the wait staff is great!',
 'great wings and decent drinks but the wait staff is awesome!',
 'great wings and decent drinks but the wait staff is amazing!']

In [ ]:
dev_data = list(open('./dev_negative'))

In [ ]:
dev_data[500:505]

['wrong !\n',
 "i 'm sure she wo n't issue a refund because - surprise !\n",
 "nothing special and the drinks are n't cheap .\n",
 'great wings and decent drinks but the wait staff is horrible !\n',
 'the tables are all saved by people crowded around the two tvs .\n']

In [ ]:
get_replacements("great wings and decent drinks but the wait staff is horrible !",
                 num_tokens=1, k_best=2)
# >>> ["great wings and decent drinks but the wait staff is great !", "great wings and decent drinks but the wait staff is awesome !"])

['great wings and decent drinks but the wait staff is great!',
 'great wings and decent drinks but the wait staff is awesome!']

__Final task__ - build a procedure that iteratively applies replacements, demonstrate the effectiveness of your approach with at least 10 examples to satisfy INGSOC.


In [ ]:
import random

In [ ]:
samples = random.sample(dev_data, 10)

In [ ]:
for sample in samples:
  print(sample)
  print(get_replacements(sample, 1, 2))
  print()

no melon ... strawberries and bananas .

['no melon... strawberries and bananas.', 'no melon... strawberry and bananas.']

despite my complaining , i continue to go back , every week ... why ?

['despite my complaining, i continue to go back, every week... why?', 'spite my complaining, i continue to go back, every week... why?']

the televisions are over _num_ years old , tube televisions with horrible viewing .

['the televisions are over _ num _ years old, tube televisions with great viewing.', 'the televisions are over _ num _ years old, tube televisions with horrible viewing.']

can you say worst customer service ever ?

['can you say worst customer service ever?', 'can you say best customer service ever?']

what the hell is that about ?

['what the hell is that about?', 'what the hell is that like?']

moreover , the salads are noticeably smaller with less fresh greens .

['moreover, the salads are much smaller with less fresh greens.', 'moreover, the salads are slightly smaller wi